In [1]:
import pymongo
import json
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["studentPerformanceF"]

with open('output.json') as f:
    data = json.load(f)

index=0
for entry in data:
    index=index+1;
    db["Student"].insert_one({
        "_id":index,
        "sex": entry["sex"],
        "age": entry["age"],
        "address": entry["address"],
        "guardian": entry["guardian"],
        "state": entry["state"],
        "health": entry["health"]
       
    })

    
    db["Skills"].insert_one({
        "_id":index,
        "math_score": entry["math_score"],
        "writing_score": entry["writing_score"],
        "reading_score": entry["reading_score"],
        "activities": entry["activities"]
        
    })

    
    db["Family"].insert_one({
        "_id":index,
        "famsize": entry["famsize"],
        "Pstatus": entry["Pstatus"],
        "mjob": entry["Mjob"],
        "fjob": entry["Fjob"],
        "famsup": entry["famsup"]
        
    })

In [2]:
from bson import json_util
def show_all_students():
    result_text.delete(1.0, tk.END)  # Clear previous content
    collection = db["Student"]
    # Retrieve all documents from the collection
    all_students = collection.find()
    # Display all student data in the result_text widget
    result_text.insert(tk.END,"The Students : "+ "\n\n")
    for student in all_students:
        pretty_student = json_util.dumps(student, indent=4)  # Pretty print the document 
        result_text.insert(tk.END, pretty_student + "\n\n")  # Insert student data into result_text
        
def show_all_skills():
    result_text.delete(1.0, tk.END)  # Clear previous content
    collection = db["Skills"]
    # Retrieve all documents from the collection
    all_skills = collection.find()
    # Display all student data in the result_text widget
    result_text.insert(tk.END,"The Students' Skills : "+ "\n\n")
    for skill in all_skills:
        pretty_skill = json_util.dumps(skill, indent=4)  # Pretty print the document
        result_text.insert(tk.END, pretty_skill + "\n\n")  # Insert student data into result_text
        

def show_all_families():
    result_text.delete(1.0, tk.END)  # Clear previous content
    collection = db["Family"]
    # Retrieve all documents from the collection
    all_families = collection.find()
    # Display all student data in the result_text widget
    result_text.insert(tk.END,"Students' Families : "+ "\n\n")
    for family in all_families:
        pretty_family = json_util.dumps(family, indent=4)  # Pretty print the document
        result_text.insert(tk.END, pretty_family + "\n\n")  # Insert student data into result_text
        
def delete_all_students():
    result_text.delete(1.0, tk.END)  # Clear previous content
    db["Student"].delete_many({})
    db["Skills"].delete_many({})
    db["Family"].delete_many({})
    str="All student deleted"
    result_text.insert(tk.END, str + "\n\n")
    
    
def delete_based_on_age():
    result_text.delete(1.0, tk.END)  # Clear previous content
    # Get the age value
    age_string = column1_vars[1].get()  # Assuming age is the second attribute in column1_vars
    
    # Convert the age value to integer
    try:
        age_value = int(age_string)
    except ValueError:
        # Handle the case where age_string cannot be converted to an integer
        result_text.insert(tk.END, "Invalid age value. Please enter a valid integer.\n\n")
        return
    
    matching_students = db["Student"].find({"age": age_string})
    
    # Retrieve the _ids of the matching documents
    student_ids = [student["_id"] for student in matching_students]
    
    result_text.insert(tk.END, f"student_ids {student_ids}.\n\n")
    
    # Delete matching documents from the "Student" collection
    db["Student"].delete_many({"_id": {"$in": student_ids}})
    
    # Delete matching documents from the "Family" collection
    db["Family"].delete_many({"_id": {"$in": student_ids}})
    
    # Delete matching documents from the "Skills" collection
    db["Skills"].delete_many({"_id": {"$in": student_ids}})
    
    result_text.insert(tk.END, f"Deleted {len(student_ids)} documents based on age : {age_value}.\n\n")
    
    
def delete_based_on_health_wri():
    result_text.delete(1.0, tk.END)  # Clear previous content
    health_string = column1_vars[5].get()
    writing_string = column2_vars[1].get()
    
    
    try:
        writing_value = int(writing_string)
    except ValueError:
        result_text.insert(tk.END, "Invalid writing value. Please enter a valid integer .\n\n")
        return
    
    try:
        health_value = int(health_string)
    except ValueError:
        result_text.insert(tk.END, "Invalid health value. Please enter a valid integer from 1 to 5 .\n\n")
        return
    
    matching_students_health = db["Student"].find({"health": health_string})
    
    student_ids__health = [student["_id"] for student in matching_students_health]
    
    matching_students_writing_health = db["Skills"].find({"$and": [{"writing_score": writing_string}, {"_id": {"$in": student_ids__health}}]})
    
    # Extracting IDs from the cursor
    matching_student_ids = [student["_id"] for student in matching_students_writing_health]
    
    db["Student"].delete_many({"_id": {"$in": matching_student_ids}})
    db["Family"].delete_many({"_id": {"$in": matching_student_ids}})
    db["Skills"].delete_many({"_id": {"$in": matching_student_ids}})
    
    result_text.insert(tk.END, f"Deleted {len(matching_student_ids)} documents based on health_string : {health_string} writing_string {writing_string}.\n\n")

    
    

def delete_based_on_state_famsup():
    result_text.delete(1.0, tk.END)  # Clear previous content
    state_string = column1_vars[4].get()
    famsup_string = column3_vars[4].get()
    
    if not state_string:
        result_text.insert(tk.END, "Invalid state value.\n\n")
        return
        
    if famsup_string not in ["no", "yes"]:
        result_text.insert(tk.END, "Invalid family support value. Please enter 'no' or 'yes'.\n\n")
        return
    
    students = db["Student"].find({"state": state_string})
    student_ids = [student["_id"] for student in students]
    
    students2 = db["Family"].find({"$and": [{"famsup": famsup_string}, {"_id": {"$in": student_ids}}]})
    student2_ids = [student["_id"] for student in students2]
    
    db["Student"].delete_many({"_id": {"$in": student2_ids}})
    db["Family"].delete_many({"_id": {"$in": student2_ids}})
    db["Skills"].delete_many({"_id": {"$in": student2_ids}})
    
    result_text.insert(tk.END, f"Deleted {len(student2_ids)} documents based on family support: {famsup_string} and state: {state_string}.\n\n")

    
    
def delete_based_on_address_activites_famSize():
    result_text.delete(1.0, tk.END)  # Clear previous content
    address= column1_vars[2].get()
    activities= column2_vars[3].get()
    famsize = column3_vars[0].get()
    
    if not address:
        result_text.insert(tk.END, "Invalid address value.\n\n")
        return
    
    if not famsize:
        result_text.insert(tk.END, "Invalid famsize value.\n\n")
        return
    
    if activities not in ["no", "yes"]:
        result_text.insert(tk.END, "Invalid activities value. Please enter 'no' or 'yes'.\n\n")
        return
    
    students = db["Student"].find({"address": address})
    student_ids = [student["_id"] for student in students]
    
    students2 = db["Family"].find({"$and": [{"famsize": famsize}, {"_id": {"$in": student_ids}}]})
    student2_ids = [student["_id"] for student in students2]
    
    students3 = db["Skills"].find({"$and": [{"activities": activities}, {"_id": {"$in":  student2_ids}}]})
    student3_ids = [student["_id"] for student in students3]
    
    db["Student"].delete_many({"_id": {"$in": student3_ids}})
    db["Family"].delete_many({"_id": {"$in": student3_ids}})
    db["Skills"].delete_many({"_id": {"$in": student3_ids}})
    
    result_text.insert(tk.END, f"Deleted {len(student2_ids)} documents based on activities: {activities},  famsize {famsize}and address: {address}.\n\n")

    
def show_female_students():
    result_text.delete(1.0, tk.END)  # Clear previous content
    collection = db["Student"]
    female_students = collection.find({"sex": "F"}, {"_id": 1, "state": 1, "guardian": 1}).sort("_id", 1)
    result_text.insert(tk.END, "Female Students:\n\n")
    for student in female_students:
        pretty_student = json_util.dumps(student, indent=4)  # Pretty print the document
        result_text.insert(tk.END, pretty_student + "\n\n")
        
        
def writing_score_index():
    collection = db['Skills']
    collection.create_index([("writing_score", 1)], unique=False)  # Create index on "writing_score" field
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, "Index on 'writing_score' field created successfully" + "\n\n")
    cursor = collection.find().sort("writing_score", 1)
    result_text.insert(tk.END, "Data Based on Writing Score:\n\n")
    for document in cursor:
        pretty_document = json_util.dumps(document, indent=4)  # Pretty print the document
        result_text.insert(tk.END, pretty_document + "\n\n")
        
def drop_writing_score_index():
    collection = db['Skills']
    index_info = collection.index_information()
    if "writing_score_1" in index_info:
        collection.drop_index("writing_score_1")
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, "Index on 'writing_score' field dropped successfully\n\n")
    else:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, "Index on 'writing_score' field does not exist\n\n")

        

def show_all_indexes():
    collection_name = 'Skills' 
    indexes = db[collection_name].list_indexes()
    
    result_text.delete(1.0, tk.END) 
    
    if indexes:
        result_text.insert(tk.END, f"Indexes for collection '{collection_name}':\n\n")
        for index in indexes:
            result_text.insert(tk.END, f"{index}\n")
    else:
        result_text.insert(tk.END, f"No indexes found for collection '{collection_name}'.\n\n")
        
        
def get_minimum():
    # Query the Skills collection to find the minimum math_score
    min_math_score_result = db["Skills"].find_one({}, sort=[("math_score", 1)])

    if min_math_score_result:
        min_math_score = min_math_score_result["math_score"]
        
        # Query the Skills collection to get IDs of documents with the minimum math_score
        skill_ids_with_min_math_score = [doc["_id"] for doc in db["Skills"].find({"math_score": min_math_score})]
        
        # Query the Student collection using the IDs from Skills collection
        students_with_min_math_score = db["Student"].find({"_id": {"$in": skill_ids_with_min_math_score}})
        
        result_text.delete(1.0, tk.END)  # Clear previous content
        
        
        result_text.insert(tk.END, f"Students with minimum math_score ({min_math_score}):\n\n")
        for student in students_with_min_math_score:
            result_text.insert(tk.END, f"{student}\n")
    else:
        result_text.delete(1.0, tk.END)  # Clear previous content
        result_text.insert(tk.END, "No records found in the Skills collection.\n\n")
        
        
def oldest_student():
    oldest_student = db["Student"].find_one(sort=[("age", -1)])

    result_text.delete(1.0, tk.END)  # Clear previous content

    if oldest_student:
        result_text.insert(tk.END, f"Oldest student information:\n\n")
        result_text.insert(tk.END, f"Address: {oldest_student['address']}\n")
        result_text.insert(tk.END, f"Age: {oldest_student['age']}\n")
        result_text.insert(tk.END, f"Gender: {oldest_student['sex']}\n")
    else:
        result_text.insert(tk.END, "No records found in the Student collection.\n\n")



def average_math():
    # Retrieve math_score values from Skills collection
    math_scores = db["Skills"].find({}, {"math_score": 1})

    # Convert math_score values to integers and calculate average
    total_math_score = 0
    count = 0
    for score in math_scores:
        try:
            math_score = int(score["math_score"])
            total_math_score += math_score
            count += 1
        except ValueError:
            # Skip if the value cannot be converted to an integer
            pass

    result_text.delete(1.0, tk.END)  # Clear previous content
    
    if count > 0:
        average = total_math_score / count
        result_text.insert(tk.END, f"Average math score: {average:.2f}\n")
    else:
        result_text.insert(tk.END, "No math scores found in the Skills collection.\n\n")

        
def distinctFjob():
    # Retrieve distinct values of Fjob attribute from Family collection
    distinct_values = db["Family"].distinct("fjob")

    result_text.delete(1.0, tk.END)  # Clear previous content
    
    if distinct_values:
        result_text.insert(tk.END, "Distinct values of Fjob:\n\n")
        for value in distinct_values[:5]:  # Display only the first 5 values
            result_text.insert(tk.END, f"{value}\n")
    else:
        result_text.insert(tk.END, "No distinct values found for Fjob in the Family collection.\n\n")
        
        
def update_math_score_by_sex():
    sex = column1_vars[0].get()
    math_score = int(column2_vars[0].get())
    students = db["Student"].find({"sex": sex})
    for student in students:
        student_id = student["_id"]
        #print(student_id)
        result = db["Skills"].update_one(
            {"_id": student_id},
            {"$set": {"math_score": math_score}})

def update_famliy_support_by_sex():
    sex = column1_vars[0].get()
    famliy_support= column3_vars[4].get()
    students = db["Student"].find({"sex": sex})
    for student in students:
        student_id = student["_id"]
        result = db["Family"].update_one(
            {"_id": student_id},
            {"$set": {"famsup": famliy_support}}
        )
def update_guardian_by_sex():
    sex = column1_vars[0].get()
    guardian= column1_vars[3].get()

    
    result = db["Student"].update_many(
        {"sex": sex},
        {"$set": {"guardian": guardian}}
    )
    print(result.modified_count)
    
    
def update_activities_by_state():
    state = column1_vars[4].get()
    activities=column2_vars[3] .get()

    students = db["Student"].find({"state": state})
    #print(students)
    modified_count=0
    
    for student in students:
        student_id = student["_id"]
        result = db["Skills"].update_one(
            {"_id": student_id},
            {"$set": {"activities": activities}}
        )  
               
        modified_count += result.modified_count
    
    print(f"Updated activities for {modified_count} students in state {state}.")
    

def update_famliy_support_by_age():
    age = column1_vars[1].get()
    famliy_support= column3_vars[4].get()
    students = db["Student"].find({"age": {"$lt": age}})
    modified_count=0
    for student in students:
        student_id = student["_id"]
        result = db["Family"].update_one(
            {"_id": student_id},
            {"$set": {"famsup": famliy_support}}
        )
        modified_count += result.modified_count
    
    print(f"Updated family support for {modified_count} students in age less then {age}.") 
    
    

def select_stu_writing_score_gt_50_famsup_yes():
    family_query = {"famsup": "yes"}
    family_result = db["Family"].find(family_query, {"_id": 1})
    
    skills_query = {
    "writing_score": {"$gte": "50"},
    "_id": {"$in": [doc["_id"] for doc in family_result]}
    }
    skills_result = db["Skills"].find(skills_query, {"_id": 1}).limit(2)
    
    stutent_query={"_id":{"$in": [doc["_id"] for doc in skills_result]}}
    student_resuit=db["Student"].find(stutent_query,{"_id":0})
    
    result_text.delete('1.0', tk.END)
    
    for doc in student_resuit:
        result_text.insert(tk.END, str(doc) + '\n')
        

        
def aggregate_female_students():
    pipeline = [
        { '$match': { "sex": "F" } },
        { '$group': { '_id': "F", 'total_female_students': { '$sum': 1 } } }
    ]
    result_text.delete('1.0', tk.END)
    result = db["Student"].aggregate(pipeline)
    for res in result:
        result_text.insert(tk.END, str(res) + '\n')
        
        
        
        
def add_stu():
    
    user_data = {
        "sex": column1_vars[0].get(),
        "age": column1_vars[1].get(),
        "address": column1_vars[2].get(),
        "guardian": column1_vars[3].get(),
        "state": column1_vars[4].get(),
        "heaith":column1_vars[5].get()
        
    }

    skills_data = {
        "math_score": column2_vars[0].get(),
        "writing_score":column2_vars[1].get(),
        "reading_score":column2_vars[2].get(),
        "activities": column2_vars[3].get()
    }

    family_data = {
        "famsize": column3_vars[0].get(),
        "Pstatus": column3_vars[1].get(),
        "Mjob": column3_vars[2].get(),
        "Fjob": column3_vars[3].get(),
        "famsup": column3_vars[4].get()
    }
    db["Student"].insert_one(user_data)
    db["Skills"].insert_one(skills_data)
    db["Family"].insert_one(family_data)
    
    for var in column1_vars:
        var.set("")
    for var in column2_vars:
        var.set("")
    for var in column3_vars:
        var.set("")

    print("Data added successfully!")
    
    
def update_math_score_by_sex():
    sex = column1_vars[0].get()
    math_score = int(column2_vars[0].get())
    students = db["Student"].find({"sex": sex})
    for student in students:
        student_id = student["_id"]
        result = db["Skills"].update_one(
            {"_id": student_id},
            {"$set": {"math_score": math_score}})



In [5]:
from bson.code import Code
import statistics


def map_reduce_sum_F_M():
    mapper = Code("""
    function () {
        emit(this.sex, 1);
    }
    """)
    reducer = Code("""
    function (key, values) {
        return Array.sum(values);
    }
    """)
    result = db.command('mapReduce', 'Student', map=mapper, reduce=reducer, out='myresult')
    result_text.delete('1.0', tk.END)
    for doc in db.myresult.find(): 
        result_text.insert(tk.END, str(doc) + '\n')
        
def map_reduce_avg_age_F_M():
    mapper = Code("""
    function () {
        emit(this.sex, {age: parseInt(this.age), count: 1});
    }
    """)

    reducer = Code("""
    function (key, values) {
        var total_age = 0;
        var total_count = 0;
        values.forEach(function(value) {
            total_age += value.age;
            total_count += value.count;
        });
        var age_avg = total_count > 0 ? total_age / total_count : 0; 
        return {age_avg: age_avg, count: total_count, age: total_age};
    }
    """)

    result = db.command('mapReduce', 'Student', map=mapper, reduce=reducer, out='myresult')

    result_text.delete('1.0', tk.END)

    for doc in db.myresult.find():
        result_text.insert(tk.END, str(doc) + '\n')   

In [6]:
import tkinter as tk
from tkinter import scrolledtext  

root = tk.Tk()

root.geometry("800x600")
root.configure(bg="#f0f0f0")

root.title("Student Performance Analysis")

canvas = tk.Canvas(root)
canvas.pack(side="left", fill="both", expand=True)


def close_window():
    root.destroy()


scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
scrollbar.pack(side="right", fill="y")
canvas.configure(yscrollcommand=scrollbar.set)

frame = tk.Frame(canvas, bg="#f0f0f0")
canvas.create_window((0, 0), window=frame, anchor="nw")

column1_attributes = ["sex", "age", "address", "guardian", "state", "health"]
column1_vars = [tk.StringVar() for _ in column1_attributes]

column2_attributes = ["math_score", "writing_score", "reading_score", "activities"]
column2_vars = [tk.StringVar() for _ in column2_attributes]

column3_attributes = ["famsize", "Pstatus", "Mjob", "Fjob", "famsup"]
column3_vars = [tk.StringVar() for _ in column3_attributes]

width_value = 15

for attribute, var in zip(column1_attributes, column1_vars):
    label = tk.Label(frame, text=attribute.capitalize(), bg="#f0f0f0", font=("Arial", 6))
    label.grid(row=column1_attributes.index(attribute), column=0, padx=3, pady=2, sticky="w")
    entry = tk.Entry(frame, textvariable=var, bg="#ffffff", font=("Arial", 6))
    entry.grid(row=column1_attributes.index(attribute), column=1, padx=3, pady=2, sticky="w")

for attribute, var in zip(column2_attributes, column2_vars):
    label = tk.Label(frame, text=attribute.capitalize(), bg="#f0f0f0", font=("Arial", 6))
    label.grid(row=column2_attributes.index(attribute), column=2, padx=3, pady=2, sticky="w")
    entry = tk.Entry(frame, textvariable=var, bg="#ffffff", font=("Arial", 6))
    entry.grid(row=column2_attributes.index(attribute), column=3, padx=3, pady=2, sticky="w")

for attribute, var in zip(column3_attributes, column3_vars):
    label = tk.Label(frame, text=attribute.capitalize(), bg="#f0f0f0", font=("Arial", 6))
    label.grid(row=column3_attributes.index(attribute), column=4, padx=3, pady=2, sticky="w")
    entry = tk.Entry(frame, textvariable=var, bg="#ffffff", font=("Arial", 6))
    entry.grid(row=column3_attributes.index(attribute), column=5, padx=3, pady=2, sticky="w")


def create_button(text, command, row, column, columnspan):
    button = tk.Button(frame, text=text, command=command, bg="#007bff", fg="#ffffff", relief=tk.FLAT, width=width_value, font=("Arial", 6))
    button.grid(row=row, column=column, columnspan=columnspan, pady=3, sticky="w")


create_button("Insert Student", add_stu, len(column1_attributes) + 1, 2, 6)
create_button("Show Student's Information", show_all_students, len(column1_attributes) + 2, 0, 4)
create_button("Show Student's Families", show_all_families, len(column1_attributes) + 2, 4, 4)
create_button("Show Student's Skills", show_all_skills, len(column1_attributes) + 3, 0, 4)
create_button("Show Female Students' Information (exculding,asc)", show_female_students, len(column1_attributes) + 3, 4, 4)
create_button("Delete All Students", delete_all_students, len(column1_attributes) + 4, 0, 4)
create_button("Delete based on age", delete_based_on_age, len(column1_attributes) + 4, 4, 4)
create_button("Delete based on writing and health", delete_based_on_health_wri, len(column1_attributes) + 5, 0, 4)
create_button("Delete based on state and family support", delete_based_on_state_famsup, len(column1_attributes) + 5, 4, 4)
create_button("Create Writing s Index", writing_score_index, len(column1_attributes) + 6, 0, 4)
create_button("Show all indexes in Skills", show_all_indexes, len(column1_attributes) + 6, 4, 4)
create_button("Drop Writing Index", drop_writing_score_index, len(column1_attributes) + 7, 0, 4)
create_button("Oldest Student", oldest_student, len(column1_attributes) + 7, 4, 4)
create_button("Get Student With Minimum Math Score", get_minimum, len(column1_attributes) + 8, 0, 4)
create_button("Average Math Score of Student", average_math, len(column1_attributes) + 8, 4, 4)
create_button("Get Distinict Fathers' jobs", distinctFjob, len(column1_attributes) + 9, 0, 4)
create_button("Aggregate Female Students", aggregate_female_students, len(column1_attributes) + 9, 4, 4)
create_button("Update Math Score by Sex", update_math_score_by_sex, len(column1_attributes) + 10, 0, 4)
create_button("Update Family Support by Sex", update_famliy_support_by_sex, len(column1_attributes) + 10, 4, 4)
create_button("Update Guardian by Sex", update_guardian_by_sex, len(column1_attributes) + 11, 0, 4)
create_button("Update Activities by State", update_activities_by_state, len(column1_attributes) + 11, 4, 4)
create_button("Update Family Support by Age", update_famliy_support_by_age, len(column1_attributes) + 12, 0, 4)
create_button("Select Students with Writing Score > 50 & Family Support Yes", select_stu_writing_score_gt_50_famsup_yes, len(column1_attributes) + 12, 4, 4)
create_button("map_reduce_sum_F_M", map_reduce_sum_F_M, len(column1_attributes) + 13, 0, 4)
create_button("map_reduce_avg_age_F_M", map_reduce_avg_age_F_M, len(column1_attributes) + 13, 4, 4)



result_text = scrolledtext.ScrolledText(root, bg="#ffffff", width=42, height=20)
result_text.pack(pady=5)

button = tk.Button(root, text="Close", command=close_window, bg="#007bff", fg="#ffffff", relief=tk.FLAT, font=("Arial", 6))
button.pack(pady=10, padx=20, ipadx=10)

def on_frame_configure(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame.bind("<Configure>", on_frame_configure)

root.mainloop()


Updated family support for 0 students in age less then .


In [7]:
import tkinter as tk
from tkinter import scrolledtext

root = tk.Tk()

root.geometry("800x600")
root.configure(bg="#f5f5dc")  # لون الخلفية بيج فاتح

root.title("Student Performance Analysis")

canvas = tk.Canvas(root, bg="#f5f5dc")  # لون خلفية الكانفس بيج فاتح
canvas.pack(side="left", fill="both", expand=True)

def close_window():
    root.destroy()

scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
scrollbar.pack(side="right", fill="y")
canvas.configure(yscrollcommand=scrollbar.set)

frame = tk.Frame(canvas, bg="#f5f5dc")
canvas.create_window((0, 0), window=frame, anchor="nw")

column1_attributes = ["sex", "age", "address", "guardian", "state", "health"]
column1_vars = [tk.StringVar() for _ in column1_attributes]

column2_attributes = ["math_score", "writing_score", "reading_score", "activities"]
column2_vars = [tk.StringVar() for _ in column2_attributes]

column3_attributes = ["famsize", "Pstatus", "Mjob", "Fjob", "famsup"]
column3_vars = [tk.StringVar() for _ in column3_attributes]

# تحسين حجم النصوص وتباعدها
def create_label_and_entry(attribute, var, row, column):
    label = tk.Label(frame, text=attribute.capitalize(), bg="#f5f5dc", font=("Arial", 12))
    label.grid(row=row, column=column, padx=5, pady=5, sticky="w")
    entry = tk.Entry(frame, textvariable=var, bg="#ffffff", font=("Arial", 12), width=20)
    entry.grid(row=row, column=column + 1, padx=5, pady=5, sticky="w")

for i, (attribute, var) in enumerate(zip(column1_attributes, column1_vars)):
    create_label_and_entry(attribute, var, i, 0)

for i, (attribute, var) in enumerate(zip(column2_attributes, column2_vars)):
    create_label_and_entry(attribute, var, i, 2)

for i, (attribute, var) in enumerate(zip(column3_attributes, column3_vars)):
    create_label_and_entry(attribute, var, i, 4)

button_styles = [
    ("#8B4513", "#ffffff"),  # brown with white text
    ("#A0522D", "#ffffff"),  # sienna with white text
    ("#D2691E", "#ffffff"),  # chocolate with white text
    ("#CD853F", "#ffffff"),  # peru with white text
    ("#F4A460", "#ffffff"),  # sandy brown with white text
    ("#DEB887", "#ffffff"),  # burlywood with white text
]

def create_button(text, command, row, column, columnspan, color_index):
    bg_color, fg_color = button_styles[color_index % len(button_styles)]
    button = tk.Button(frame, text=text, command=command, bg=bg_color, fg=fg_color, relief=tk.FLAT, font=("Arial", 10), padx=10, pady=5)
    button.grid(row=row, column=column, columnspan=columnspan, pady=5, padx=5, sticky="ew")

buttons = [
    ("Insert Student", add_stu),
    ("Show Student's Information", show_all_students),
    ("Show Student's Families", show_all_families),
    ("Show Student's Skills", show_all_skills),
    ("Show Female Students' Information (excluding,asc)", show_female_students),
    ("Delete All Students", delete_all_students),
    ("Delete based on age", delete_based_on_age),
    ("Delete based on writing and health", delete_based_on_health_wri),
    ("Delete based on state and family support", delete_based_on_state_famsup),
    ("Create Writing s Index", writing_score_index),
    ("Show all indexes in Skills", show_all_indexes),
    ("Drop Writing Index", drop_writing_score_index),
    ("Oldest Student", oldest_student),
    ("Get Student With Minimum Math Score", get_minimum),
    ("Average Math Score of Student", average_math),
    ("Get Distinct Fathers' jobs", distinctFjob),
    ("Aggregate Female Students", aggregate_female_students),
    ("Update Math Score by Sex", update_math_score_by_sex),
    ("Update Family Support by Sex", update_famliy_support_by_sex),
    ("Update Guardian by Sex", update_guardian_by_sex),
    ("Update Activities by State", update_activities_by_state),
    ("Update Family Support by Age", update_famliy_support_by_age),
    ("Select Students with Writing Score > 50 & Family Support Yes", select_stu_writing_score_gt_50_famsup_yes),
    ("Map Reduce Sum F_M", map_reduce_sum_F_M),
    ("Map Reduce Avg Age F_M", map_reduce_avg_age_F_M)
]

for i, (text, command) in enumerate(buttons):
    create_button(text, command, len(column1_attributes) + 1 + (i // 2), (i % 2) * 4, 4, i)

result_text = scrolledtext.ScrolledText(root, bg="#ffffff", font=("Arial", 12), width=50, height=20)
result_text.pack(pady=5, padx=10, fill="both", expand=True)

button = tk.Button(root, text="Close", command=close_window, bg="#8B4513", fg="#ffffff", relief=tk.FLAT, font=("Arial", 12), padx=20, pady=10)
button.pack(pady=10, padx=20, ipadx=10)

def on_frame_configure(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame.bind("<Configure>", on_frame_configure)

root.mainloop()
